In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lazypredict
from lazypredict.Supervised import LazyRegressor

Đọc và chia data

In [25]:
from scipy import stats
data = pd.read_csv("dataAfterCleaningFinal.csv", delimiter=",")
data.head()

,chieuDai,chieuNgang,dienTich,Gia/m2,Phongngu,SoTang,PhongTam,Loai,GiayTo,TinhTrangNoiThat,QuanHuyen
0,20.00,5.00,100.00,604.17,3,1.00,3.00,"nhà ngõ, hẻm",đã có sổ,nội thất đầy đủ,quận 8
1,16.00,5.00,80.00,1718.75,2,1.00,2.00,"nhà ngõ, hẻm",đã có sổ,NaN,quận 12
2,15.00,8.00,117.40,4542.92,8,5.00,7.00,nhà biệt thự,đã có sổ,hoàn thiện cơ bản,quận gò vấp
3,32.00,3.20,102.00,2982.08,3,2.00,2.00,"nhà mặt phố, mặt tiền",đã có sổ,nội thất đầy đủ,quận tân phú
4,4.00,6.00,26.00,4295.00,2,2.00,3.00,"nhà ngõ, hẻm",đã có sổ,nội thất đầy đủ,quận phú nhuận


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17589 entries, 0 to 17588
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   chieuDai          14772 non-null  float64
 1   chieuNgang        15077 non-null  float64
 2   dienTich          17589 non-null  float64
 3   Gia/m2            17589 non-null  float64
 4   Phongngu          17589 non-null  int64  
 5   SoTang            13463 non-null  float64
 6   PhongTam          13614 non-null  float64
 7   Loai              17589 non-null  object 
 8   GiayTo            17526 non-null  object 
 9   TinhTrangNoiThat  9107 non-null   object 
 10  QuanHuyen         17589 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.5+ MB


In [27]:
target = "Gia/m2"
x = data.drop(target, axis=1)
y = data[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

In [28]:
print("train.shape: ",x_train.shape)
print("test.shape: ",x_test.shape)

train.shape:  (14071, 10)
test.shape:  (3518, 10)


Xem unique của từng cột categorical

In [16]:
print("uniqueLoai:")
x_train.Loai.unique()

uniqueLoai:


array(['nhà ngõ, hẻm', 'nhà mặt phố, mặt tiền', 'nhà phố liền kề',
       'nhà biệt thự'], dtype=object)

In [7]:
print("uniqueGiayTo:")
x_train.GiayTo.unique()

uniqueGiayTo:


array(['đã có sổ', nan, 'sổ chung / công chứng vi bằng', 'không có sổ',
       'giấy tờ viết tay', 'đang chờ sổ'], dtype=object)

In [8]:
print("uniqueTinhTrangNoiThat:")
x_train.TinhTrangNoiThat.unique()

uniqueTinhTrangNoiThat:


array([nan, 'nội thất cao cấp', 'nội thất đầy đủ', 'bàn giao thô',
       'hoàn thiện cơ bản'], dtype=object)

In [17]:
print("uniqueQuanHuyen:")
x_train.QuanHuyen.unique()

uniqueQuanHuyen:


array(['quận tân phú', 'quận 12', 'quận gò vấp', 'quận 11',
       'quận phú nhuận', 'quận 6', 'quận bình tân', 'quận 4',
       'quận tân bình', 'quận bình thạnh', 'quận 7', 'quận 1',
       'huyện hóc môn', 'huyện bình chánh', 'quận 8', 'quận 5', 'quận 3',
       'huyện nhà bè', 'quận 10', 'huyện củ chi'], dtype=object)

In [29]:
## lấy ra list quận huyện theo thứ tự tăng dần của giá
df = pd.DataFrame(data)

def sorted_districts(col):
    table = df.groupby(col)['Gia/m2'].mean().reset_index()
  
    sorted_table = table.sort_values(by='Gia/m2')
    sorted_districts = sorted_table[col].tolist()
    return sorted_districts

list_sorted_districts = sorted_districts('QuanHuyen')

In [19]:
for i in range(0, len(list_sorted_districts), 4):
    print(list_sorted_districts[i:i+4])

['huyện củ chi', 'huyện bình chánh', 'huyện hóc môn', 'huyện nhà bè']
['quận 12', 'quận bình tân', 'quận tân phú', 'quận 7']
['quận gò vấp', 'quận 6', 'quận bình thạnh', 'quận 4']
['quận 8', 'quận tân bình', 'quận 11', 'quận phú nhuận']
['quận 10', 'quận 3', 'quận 5', 'quận 1']


Tiền xử lý lần 2

In [30]:
Loai_values = ["unknown",'nhà ngõ, hẻm', 'nhà phố liền kề', 'nhà mặt phố, mặt tiền', 'nhà biệt thự']
GiayTo_values = ["unknown",'giấy tờ viết tay', 'không có sổ', 'đang chờ sổ', 
            'sổ chung / công chứng vi bằng', 'đã có sổ']
TinhTrangNoiThat_values = ["unknown",'bàn giao thô', 'hoàn thiện cơ bản', 'nội thất đầy đủ', 'nội thất cao cấp']
QuanHuyen_values = list_sorted_districts

ord_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
    ("encoder", OrdinalEncoder(categories=[Loai_values,GiayTo_values,TinhTrangNoiThat_values,QuanHuyen_values]))
])

num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
preprocessor = ColumnTransformer(transformers=[
    ("num_features", num_transformer, ["chieuDai", "chieuNgang","dienTich","Phongngu","SoTang","PhongTam"]),
    ("ordinal_features", ord_transformer, ["Loai","GiayTo","TinhTrangNoiThat","QuanHuyen"]),
])



In [14]:
x_train_preprocessor = preprocessor.fit_transform(x_train)
x_test_preprocessor = preprocessor.transform(x_test)
lazy_reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None )
models, predictions = lazy_reg.fit(x_train_preprocessor, x_test_preprocessor, y_train, y_test)
print(predictions)

100%|██████████| 42/42 [01:58<00:00,  2.81s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001452 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 774
[LightGBM] [Info] Number of data points in the train set: 14071, number of used features: 41
[LightGBM] [Info] Start training from score 4769.921673
                                       Adjusted R-Squared  \
Model                                                       
DecisionTreeRegressor                                0.87   
XGBRegressor                                         0.82   
ExtraTreesRegressor                                  0.78   
BaggingRegressor                                     0.74   
RandomForestRegressor                                0.73   
ExtraTreeRegressor                                   0.68   
LGBMRegressor                                        0.27   
MLPRegressor                               

In [31]:
from sklearn.linear_model import PoissonRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
reg = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", ExtraTreesRegressor()),
])
reg.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_features',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['chieuDai', 'chieuNgang',
                                                   'dienTich', 'Phongngu',
                                                   'SoTang', 'PhongTam']),
                                                 ('ordinal_features',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('enco...
                                                                                               'huyện '
                                                                                               'nhà '
                                                                                               'bè',
                                                                                               'quận '
                                                                                               'bình '
                                                                                               'tân',
                                                                                               '11 '
                                                                                               'ngô '
                                                                                               'gia '
                                                                                               'tự '
                                                                                               'phường '
                                                                                               '2 '
                                                                                               'quận '
                                                                                               '10',
                                                                                               'quận '
                                                                                               'tân '
                                                                                               'phú',
                                                                                               'quận '
                                                                                               'gò '
                                                                                               'vấp',
                                                                                               'quận '
                                                                                               '7',
                                                                                               'quận '
                                                                                               '8',
                                                                                               'quận '
                                                                                               '6',
                                                                                               'quận '
                                                                                               'bình '
                                        

In [32]:
y_predict = reg.predict(x_test)

In [33]:
accuracy_list = [(i, j, abs(i - j) / i * 100 if i != 0 else 0) for i, j in zip(y_test, y_predict)]

accuracy_list_sorted = sorted(accuracy_list, key=lambda x: x[2])

for actual, predict, percentage_error in accuracy_list_sorted:
    print("Actual: {} - Predict: {} - Percentage Error: {:.2f}%".format(actual, predict, percentage_error))



Actual: 3255.0 - Predict: 3255.0 - Percentage Error: 0.00%
Actual: 4192.5 - Predict: 4192.5 - Percentage Error: 0.00%
Actual: 3273.75 - Predict: 3273.75 - Percentage Error: 0.00%
Actual: 2675.0 - Predict: 2675.0 - Percentage Error: 0.00%
Actual: 5922.5 - Predict: 5922.5 - Percentage Error: 0.00%
Actual: 1250.0 - Predict: 1250.0 - Percentage Error: 0.00%
Actual: 1736.25 - Predict: 1736.25 - Percentage Error: 0.00%
Actual: 5165.0 - Predict: 5165.0 - Percentage Error: 0.00%
Actual: 3962.5 - Predict: 3962.5 - Percentage Error: 0.00%
Actual: 8433.75 - Predict: 8433.75 - Percentage Error: 0.00%
Actual: 7647.5 - Predict: 7647.5 - Percentage Error: 0.00%
Actual: 3338.75 - Predict: 3338.75 - Percentage Error: 0.00%
Actual: 6210.0 - Predict: 6210.0 - Percentage Error: 0.00%
Actual: 2562.5 - Predict: 2562.5 - Percentage Error: 0.00%
Actual: 2956.25 - Predict: 2956.25 - Percentage Error: 0.00%
Actual: 5090.0 - Predict: 5090.0 - Percentage Error: 0.00%
Actual: 486.25 - Predict: 486.25 - Percentage 

In [34]:
print("MAE {}".format(mean_absolute_error(y_test, y_predict)))
print("MSE {}".format(mean_squared_error(y_test, y_predict)))
print("R2 {}".format(r2_score (y_test, y_predict)))

MAE 944.2054549862864
MSE 6925995.819351901
R2 0.8751899679735782
